**Downloading Scripts, generated by AI**

---


This downloads the clips in form of .m4a file. For downloading the top clips, we use 5 year data, for each week we get the top 100 clips, and run this for the entire time duration, and then sort by views, and then download the required amount of clips. MultiThreading is used for parallel downloading. For downloading the negative dataset, since VODs are cleared out after max of 60 days by Twitch, we sort by views the top available VODs, get metadata of the top clips of that stream, and download around 50 clips of 30 second each, avoiding the clipped regions. the outputs are moved to gdrive in batches, each batch contains a metadata file containing the specifications of each and every clip- including view count streamer_id, start time, end time, game id. This is possible through Twitch API, which requires the ID and secret code, generated through twitch developers console.

Mounting Google Drive for Access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Dependencies

In [ ]:
!apt-get -y install ffmpeg
!pip install yt-dlp requests tqdm

Downloading positive clips (i had downloaded game wise, here is the code which i had used for CS2 clips, have done similar for Apex Legends and Valorant)

In [ ]:
import requests
import os
import json
import shutil
import time
import logging
import yt_dlp
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Tuple, Optional

# ==============================================================================
# CONFIGURATION
# ==============================================================================
class Config:
    """Central configuration for the dataset collector."""
    # Twitch API Credentials
    CLIENT_ID = '4ursdzwlj6dbb35hxij9cnooap044r'
    CLIENT_SECRET = '4g8dimywmzmzw1e2aja3uabagvkxv2'

    # Target Game: Counter-Strike (CS:GO & CS2)
    GAME_ID = "32399"

    # Data Collection Goals
    TOTAL_TARGET_CLIPS = 12000
    LOOKBACK_YEARS = 5

    # Batch Processing Settings
    BATCH_SIZE = 500         # Number of clips to process before zipping/uploading
    MAX_WORKERS = 16         # Number of parallel download threads

    # Directories (optimized for Google Colab)
    BASE_DIR = "/content/cs2_audio_batches"
    DRIVE_FOLDER = "/content/drive/MyDrive/CS2_Dataset_12k"

# Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

# ==============================================================================
# MAIN CLASS
# ==============================================================================
class TwitchAudioCollector:
    def __init__(self):
        """Initialize directories and setup."""
        self.headers = None
        self._setup_directories()

    def _setup_directories(self):
        """Ensures the output directories exist."""
        if not os.path.exists(Config.DRIVE_FOLDER):
            os.makedirs(Config.DRIVE_FOLDER)
            logger.info(f"Created Drive folder: {Config.DRIVE_FOLDER}")

        if not os.path.exists(Config.BASE_DIR):
            os.makedirs(Config.BASE_DIR)

    # --------------------------------------------------------------------------
    # 1. AUTHENTICATION
    # --------------------------------------------------------------------------
    def authenticate(self):
        """
        Obtains an OAuth 2.0 Access Token from Twitch.
        Raises an exception if authentication fails.
        """
        url = "https://id.twitch.tv/oauth2/token"
        params = {
            "client_id": Config.CLIENT_ID,
            "client_secret": Config.CLIENT_SECRET,
            "grant_type": "client_credentials"
        }

        try:
            response = requests.post(url, params=params)
            response.raise_for_status() # Raise error for 4xx/5xx responses
            data = response.json()

            self.headers = {
                "Client-ID": Config.CLIENT_ID,
                "Authorization": f"Bearer {data['access_token']}"
            }
            logger.info("✅ Twitch Authentication Successful")
        except Exception as e:
            logger.error(f"Authentication Failed: {e}")
            raise

    # --------------------------------------------------------------------------
    # 2. METADATA SCANNING
    # --------------------------------------------------------------------------
    def fetch_historical_metadata(self) -> List[Dict]:
        """
        Performs a 'Smart Scan' of the last N years.
        Because Twitch API doesn't allow sorting by 'all time' deep in the past,
        we iterate through small time windows (7 days) to gather high-density data.
        """
        logger.info(f"--- Step 1: Scanning {Config.LOOKBACK_YEARS} Years of History ---")

        clips = []
        end_date = datetime.now(timezone.utc)
        start_date = end_date - timedelta(days=365 * Config.LOOKBACK_YEARS)

        current_window_start = start_date
        total_weeks = int((end_date - start_date).days / 7)

        # tqdm creates a visual progress bar
        with tqdm(total=total_weeks, desc="Scanning Timeline", unit="week") as pbar:
            while current_window_start < end_date:
                # Stop if we have enough raw candidates (1.5x buffer for deduplication)
                if len(clips) > (Config.TOTAL_TARGET_CLIPS * 1.5):
                    break

                # Define a 7-day window
                window_end = min(current_window_start + timedelta(days=7), end_date)

                params = {
                    "game_id": Config.GAME_ID,
                    "started_at": current_window_start.isoformat(),
                    "ended_at": window_end.isoformat(),
                    "first": 100  # Max allowed by Twitch per request
                }

                try:
                    response = requests.get(
                        "https://api.twitch.tv/helix/clips",
                        headers=self.headers,
                        params=params
                    )
                    if response.status_code == 200:
                        batch = response.json().get("data", [])
                        clips.extend(batch)
                except Exception as e:
                    logger.warning(f"Failed to fetch window {current_window_start}: {e}")

                # Move to next week
                current_window_start = window_end
                pbar.update(1)

                # Tiny sleep to be kind to the API rate limit
                time.sleep(0.1)

        # ----------------------------------------------------------------------
        # Data Cleaning: Deduplication & Sorting
        # ----------------------------------------------------------------------
        # Use a dictionary comprehension to remove duplicates based on Clip ID
        unique_clips_map = {c['id']: c for c in clips}
        unique_clips = list(unique_clips_map.values())

        # Sort by view count (Highest to Lowest) to get the best content
        sorted_clips = sorted(unique_clips, key=lambda x: x["view_count"], reverse=True)

        # Trim to exact target
        final_list = sorted_clips[:Config.TOTAL_TARGET_CLIPS]

        logger.info(f"--- Metadata Done: Found {len(final_list)} unique top clips ---")
        return final_list

    # --------------------------------------------------------------------------
    # 3. DOWNLOADING LOGIC (Static Method for Threading)
    # --------------------------------------------------------------------------
    @staticmethod
    def _download_worker(args: Tuple) -> Tuple[bool, Dict]:
        """
        Worker function for ThreadPoolExecutor.
        Downloads audio for a single clip using yt-dlp.

        Args:
            args: Tuple containing (clip_metadata, save_directory)

        Returns:
            Tuple: (Success_Boolean, Clip_Metadata_or_Error_Message)
        """
        clip, save_dir = args
        try:
            # Create a filesystem-safe filename
            # Remove special chars, keep alphanumeric, spaces, dashes
            safe_title = "".join([c for c in clip['title'] if c.isalnum() or c in (' ','-','_')])
            safe_title = safe_title.strip()[:40] # Truncate to 40 chars

            filename_base = f"V{clip['view_count']}_{safe_title}_{clip['id']}"
            out_path_template = os.path.join(save_dir, filename_base)

            # Check if already exists (resume capability)
            if os.path.exists(out_path_template + ".m4a"):
                return (True, clip)

            # Configure yt-dlp for high-speed audio extraction
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'm4a', # m4a is faster/smaller than mp3 for this
                }],
                'outtmpl': out_path_template,
                'quiet': True,
                'no_warnings': True,
            }

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([clip['url']])

            return (True, clip)

        except Exception as e:
            return (False, f"Clip {clip.get('id', 'Unknown')} Failed: {str(e)}")

    # --------------------------------------------------------------------------
    # 4. BATCH PROCESSING PIPELINE
    # --------------------------------------------------------------------------
    def process_and_upload(self, all_clips: List[Dict]):
        """
        Splits metadata into batches, downloads them, zips them,
        uploads to Google Drive, and cleans up local storage.
        """
        total_batches = (len(all_clips) // Config.BATCH_SIZE) + 1

        logger.info(f"\n--- Starting Processing: {len(all_clips)} clips in {total_batches} batches ---")
        logger.info(f"📂 Target Drive Folder: {Config.DRIVE_FOLDER}")

        for i in range(total_batches):
            batch_start = i * Config.BATCH_SIZE
            batch_end = min((i + 1) * Config.BATCH_SIZE, len(all_clips))

            if batch_start >= batch_end:
                break

            batch_clips = all_clips[batch_start:batch_end]
            batch_num = i + 1

            # A. Prepare Batch Directory
            batch_dir_name = f"batch_{batch_num}"
            batch_dir_path = os.path.join(Config.BASE_DIR, batch_dir_name)

            # Ensure clean slate
            if os.path.exists(batch_dir_path):
                shutil.rmtree(batch_dir_path)
            os.makedirs(batch_dir_path)

            logger.info(f"Processing Batch {batch_num}/{total_batches} (Clips {batch_start}-{batch_end})")

            # B. Parallel Download
            download_args = [(clip, batch_dir_path) for clip in batch_clips]
            successful_clips = []

            # Use ThreadPoolExecutor for concurrent downloads
            with ThreadPoolExecutor(max_workers=Config.MAX_WORKERS) as executor:
                results = list(tqdm(
                    executor.map(self._download_worker, download_args),
                    total=len(batch_clips),
                    unit="clip",
                    leave=False
                ))

            for success, data in results:
                if success:
                    successful_clips.append(data)
                # Optional: Log failures here if needed

            # C. Save Batch Metadata
            # We save a JSON manifest so we know exactly what is in this zip
            manifest = {c['id']: c for c in successful_clips}
            with open(os.path.join(batch_dir_path, "metadata.json"), "w") as f:
                json.dump(manifest, f, indent=4)

            # D. Zip & Upload
            zip_filename = f"CS2_Batch_{batch_num}_{len(successful_clips)}clips"
            zip_path_no_ext = os.path.join("/content", zip_filename) # make_archive adds .zip

            shutil.make_archive(zip_path_no_ext, 'zip', batch_dir_path)

            final_zip_path = zip_path_no_ext + ".zip"
            drive_dest_path = os.path.join(Config.DRIVE_FOLDER, f"{zip_filename}.zip")

            logger.info(f"⬆️ Uploading Batch {batch_num} to Drive...")
            shutil.move(final_zip_path, drive_dest_path)

            # E. Cleanup
            # Remove the unzipped folder to save Colab disk space
            shutil.rmtree(batch_dir_path)

            logger.info(f"✅ Batch {batch_num} Completed & Cleaned.")

# ==============================================================================
# ENTRY POINT
# ==============================================================================
if __name__ == "__main__":
    start_time = time.time()

    collector = TwitchAudioCollector()

    try:
        # 1. Login
        collector.authenticate()

        # 2. Get Data
        all_clips_metadata = collector.fetch_historical_metadata()

        # 3. Download & Upload
        if all_clips_metadata:
            collector.process_and_upload(all_clips_metadata)
        else:
            logger.warning("No clips found to download.")

    except KeyboardInterrupt:
        logger.info("\n🛑 Process interrupted by user.")
    except Exception as e:
        logger.error(f"Critical Error: {e}")
    finally:
        duration = (time.time() - start_time) / 60
        logger.info(f"--- Script Finished in {duration:.2f} minutes ---")

Downloading negative clips (i had downloaded Valorant seperately and CS2 and APex seperately, for testing purposes), this is the code for downloading CS2 and Apex Files

In [ ]:
import requests
import os
import shutil
import json
import time
import zipfile
import subprocess
import yt_dlp
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# ================= SETTINGS =================
# My Twitch Keys (don't share these!)
CLIENT_ID = '4ursdzwlj6dbb35hxij9cnooap044r'
CLIENT_SECRET = '4g8dimywmzmzw1e2aja3uabagvkxv2'

# How many clips do we want?
TOTAL_GOAL = 32000
CLIPS_PER_VID = 3

# Batch settings
BATCH_SIZE = 350
WORKERS = 8  # Number of threads to run at once
CLIP_LENGTH = 30 # seconds

# Folders
DRIVE_PATH = "/content/drive/MyDrive/Negative_Dataset_48k"
TEMP_FOLDER = "/content/temp_mining_zone"

# Create the main folders if they don't exist
if not os.path.exists(os.path.join(DRIVE_PATH, "CS2_Set")):
    os.makedirs(os.path.join(DRIVE_PATH, "CS2_Set"))

if not os.path.exists(os.path.join(DRIVE_PATH, "Apex_Set")):
    os.makedirs(os.path.join(DRIVE_PATH, "Apex_Set"))

# File that has all the links we need to download
INPUT_FILE = os.path.join(DRIVE_PATH, "download_manifest_split.json")

# This set will hold IDs of videos that failed so we don't try them again in other threads
BROKEN_LINKS = set()

# ================= HELPER FUNCTIONS =================

def get_access_token():
    # Need this to talk to Twitch API
    url = f"https://id.twitch.tv/oauth2/token?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&grant_type=client_credentials"
    try:
        resp = requests.post(url).json()
        return {"Client-ID": CLIENT_ID, "Authorization": f"Bearer {resp['access_token']}"}
    except:
        print("Error getting token")
        return None

def load_the_list():
    # Reads the big json file with all the download tasks
    if os.path.exists(INPUT_FILE):
        with open(INPUT_FILE, 'r') as f:
            return json.load(f)
    else:
        print("Could not find the manifest file!")
        return []

# ================= THE MAIN WORKER =================

def process_one_video(task, save_folder):
    # Check if this video is already known to be broken
    if task['vod_id'] in BROKEN_LINKS:
        return []

    # File names
    raw_name = f"raw_{task['filename']}"
    raw_full_path = os.path.join(save_folder, raw_name + ".m4a")
    final_name_base = os.path.join(save_folder, task['filename'])

    # List to store info about clips we make
    clips_info = []

    # Settings for the downloader (yt-dlp)
    # I found these settings online to make it faster
    dl_options = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'm4a'}],
        'postprocessor_args': ['-t', '90'], # Download 90 seconds
        # This lambda part tells it specifically where to start downloading
        'download_ranges': lambda info, ydl: [{'start_time': task['start'], 'end_time': task['end']}],
        'outtmpl': f"{os.path.join(save_folder, raw_name)}.%(ext)s",
        'quiet': True,
        'no_warnings': True,
        'ignoreerrors': True,
        'socket_timeout': 10,
        'retries': 2
    }

    try:
        # Try to download
        with yt_dlp.YoutubeDL(dl_options) as ydl:
            ydl.download([task['vod_url']])

        # Check if the file actually downloaded
        if os.path.exists(raw_full_path) and os.path.getsize(raw_full_path) > 1000:

            # If it worked, split it into 3 smaller parts using ffmpeg
            for i in range(CLIPS_PER_VID):
                part_filename = f"{final_name_base}_part{i+1}.m4a"
                start_time = i * CLIP_LENGTH

                # FFMPEG command to chop the audio
                command = [
                    'ffmpeg', '-y', '-v', 'error', '-i', raw_full_path,
                    '-ss', str(start_time), '-t', str(CLIP_LENGTH), '-c', 'copy', part_filename
                ]

                try:
                    subprocess.run(command, timeout=20)

                    # If split worked, save the info
                    if os.path.exists(part_filename):
                        clips_info.append({
                            "filename": os.path.basename(part_filename),
                            "label": 0, # 0 means no exciting event
                            "source_vod": task['vod_id'],
                            "timestamp": task['start'] + start_time
                        })
                except:
                    pass # Just skip if one chop fails

            # Delete the big raw file to save space
            os.remove(raw_full_path)
            return clips_info

        else:
            # If download failed, add to bad list
            BROKEN_LINKS.add(task['vod_id'])
            return []

    except Exception as e:
        # If anything else crashes, just mark it as broken
        BROKEN_LINKS.add(task['vod_id'])
        return []

def run_a_batch(batch_number, tasks_in_batch):
    # Figure out which game we are doing
    game_name = tasks_in_batch[0]['game']

    # Decide where to save the zip file
    if game_name == "CS2":
        dest_folder = os.path.join(DRIVE_PATH, "CS2_Set")
        zip_prefix = "cs2_batch"
    else:
        dest_folder = os.path.join(DRIVE_PATH, "Apex_Set")
        zip_prefix = "apex_batch"

    print(f"\nStarting Batch {batch_number} for {game_name}...")

    # Create a temp folder for this batch
    current_batch_folder = os.path.join(TEMP_FOLDER, f"batch_{batch_number}")
    if os.path.exists(current_batch_folder):
        shutil.rmtree(current_batch_folder)
    os.makedirs(current_batch_folder)

    all_metadata = []

    # Run the downloads in parallel threads
    with ThreadPoolExecutor(max_workers=WORKERS) as executor:
        # Submit all tasks
        futures = [executor.submit(process_one_video, t, current_batch_folder) for t in tasks_in_batch]

        # Show progress bar
        for f in tqdm(as_completed(futures), total=len(tasks_in_batch), desc=f"Batch {batch_number}", leave=False):
            result = f.result()
            if result:
                all_metadata.extend(result)

    # Save the metadata json for this batch
    with open(os.path.join(current_batch_folder, "metadata.json"), 'w') as f:
        json.dump(all_metadata, f)

    # Zip everything up
    zip_filename = f"{zip_prefix}_{batch_number}.zip"
    zip_path = os.path.join(TEMP_FOLDER, zip_filename)

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(current_batch_folder):
            for file in files:
                zipf.write(os.path.join(root, file), file)

    # Upload to Google Drive
    print(f"Uploading {zip_filename} to Drive...", flush=True)
    shutil.copy2(zip_path, os.path.join(dest_folder, zip_filename))

    # Cleanup
    shutil.rmtree(current_batch_folder)
    os.remove(zip_path)
    print(f"Batch {batch_number} finished. Broken VODs ignored: {len(BROKEN_LINKS)}")

# ================= START HERE =================

if __name__ == "__main__":

    # Load the big list of things to do
    full_list = load_the_list()

    # Split list by game
    cs2_list = [x for x in full_list if x['game'] == 'CS2']
    apex_list = [x for x in full_list if x['game'] == 'Apex']

    # --- DO CS2 BATCHES ---
    print(f"\nDoing CS2 tasks ({len(cs2_list)} items)...")

    # Calculate how many batches we need
    num_batches_cs2 = (len(cs2_list) + BATCH_SIZE - 1) // BATCH_SIZE

    for i in range(num_batches_cs2):
        # Check if we already did this one
        zip_check = f"cs2_batch_{i}.zip"
        path_check = os.path.join(DRIVE_PATH, "CS2_Set", zip_check)

        if os.path.exists(path_check) and os.path.getsize(path_check) > 1000:
            print(f"Skipping Batch {i} (Already done)")
            continue

        # Get the slice of tasks for this batch
        start_index = i * BATCH_SIZE
        end_index = min((i + 1) * BATCH_SIZE, len(cs2_list))
        batch_tasks = cs2_list[start_index:end_index]

        run_a_batch(i, batch_tasks)

    # --- DO APEX BATCHES ---
    print(f"\nDoing Apex tasks ({len(apex_list)} items)...")

    num_batches_apex = (len(apex_list) + BATCH_SIZE - 1) // BATCH_SIZE

    for i in range(num_batches_apex):
        zip_check = f"apex_batch_{i}.zip"
        path_check = os.path.join(DRIVE_PATH, "Apex_Set", zip_check)

        if os.path.exists(path_check) and os.path.getsize(path_check) > 1000:
            print(f"Skipping Batch {i} (Already done)")
            continue

        start_index = i * BATCH_SIZE
        end_index = min((i + 1) * BATCH_SIZE, len(apex_list))
        batch_tasks = apex_list[start_index:end_index]

        run_a_batch(i, batch_tasks)

    print("All done!")

After downloading all this data, we have to run it through Demucs,